In [ ]:
import numpy as np
from sklearn.metrics import ndcg_score 

In [ ]:
class AdaRank():
    def __init__(self, k = 10, E = ndcg_score, n_iterations = 1000, ):
        self.n_iterations = n_iterations
        self.E = E
        self.k = k
        self.weights = None
        self.weak_ranks = []
        self.alphas = []
    
    def fit(self, X, y, qid):
        
        n_queries = np.unique(qid).shape[0]
        self.weights = np.ones(n_queries /n_queries)
        
        for i in range(self.n_iterations):
            
            # weak rank
            # h_predictions = []
            # for i, x in enumerate(X):
            #     h_prediction = self._weak_rank(x)
            #     h_predictions.append(h_prediction)
            
            h_predictions = self._weak_rank(X)
            
            # h_predictions = weak_ranker(q, d, weights)
            h_score = self.E(y, h_predictions, k = self.k)
            
            # Choose alpha t
            alpha = 0.5 * np.log((weights * (1 + h_score)) / (1 - h_score))
            self.alphas.append(alpha)
            
            # f_predictions = []
            # for i, x in enumerate(X):
            #     f_prediction = np.sum(self.alphas * self.weak_ranks[:,i])
            #     f_predictions.append(f_prediction)
            
            f_predictions = np.sum(self.alphas * self.weak_ranks, axis = 0)
            
            # update P t+1
            f_score = self.E(y, f_predictions, k = self.k)
            weights = np.exp(-f_score) / np.exp(np.sum(-f_score))
    
    def _weak_rank(self,X):
        """Create weak ranker ht with weighted distribution Pt on training data"""
        
        # Calculate the probabilities of selecting each query based on the current weights
        query_probabilities = self.weights / np.sum(self.weights)
        
        #TODO: CREATE WEAK RANKER
        
        pass
            

In [4]:
import numpy as np
from sklearn.metrics import ndcg_score
# we have groud-truth relevance of some answers to a query:
true_relevance = np.asarray([[10, 0, 0, 1, 5]])
# we predict some scores (relevance) for the answers
scores = np.asarray([[.1, .2, .3, 4, 70]])
ndcg_score(true_relevance, scores, k = 2)

0.4280562600295606

In [8]:
q = np.asarray([[1, 2, 3], [4, 5, 6]])
d = np.asarray([[[7, 8, 9],[1, 2, 3],[3,4,5]], [[10, 11, 12],[1, 2, 3],[3,4,5]]])
X = np.concatenate((q, d), axis = 1) # TODO check 

q = np.asarray([[1, 2, 3], [4, 5, 6]])
d = np.asarray([[[7, 8, 9],[1, 2, 3],[3,4,5]], [[10, 11, 12],[1, 2, 3],[3,4,5]]])
r = np.asarray([1,2,3])

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 3 dimension(s)